# Annexe D : Modèle d'implémentation des forces

Pour faciliter la formalisation des différentes forces, nous définissons une classe abstraite `Force` qui guide le fonctionnement du modèle d'application des forces. Pour chaque force $\vec F$, en application du principe fondamental de la dynamique $\vec F=m\vec a$, nous définissons $\ddot r=\dfrac{\vec F}m$ la contribution de la force à l'accération d'une particule de masse $m$.

Chaque classe héritant de `Force` doit définir les méthodes abstraites de définition de nom, formule et accélération de la force qu'elle définit.
La classe abstraite définit les méthodes de représentation par défaut des classes filles, ainsi que des constantes usuelles dans les formules des forces de notre modèle, à savoir $G=6.67430\cdot10^{-20}\mathrm{~km}^3\mathrm{~kg}^{-1}\mathrm{~s}^{-2}$ la constante universelle de gravitation, $M=5.972\cdot10^{24}\mathrm{~kg}$ la masse de la Terre et $R=6371\mathrm{~km}$ son rayon.

In [ ]:
from abc import ABC, abstractmethod
import numpy as np

class Force(ABC):
    """Classe abstraite de modélisation d'une force."""

    # Le décorateur property permet de définir une propriété en lecture-seule.
    @property
    def G(self):
        """Constante universelle de gravitation."""
        return 6.67430e-20  # km^3 kg^−1 s^−2

    @property
    def earth_mass(self):
        """Masse de la Terre."""
        return 5.972e24  # kg

    @property
    def earth_radius(self):
        """Rayon moyen volumétrique de la Terre."""
        return 6371  # km

    @property
    @abstractmethod
    def name(self):
        """Nom de la force sous forme de string."""
        raise NotImplementedError(self.name)
    
    @property
    @abstractmethod
    def formula(self):
        """Expression de la force au format LaTeX."""
        raise NotImplementedError(self.formula)
    
    @abstractmethod
    def acceleration(self, u, t):
        """Accélération de la force appliquée au vecteur spécifié."""
        raise NotImplementedError(self.acceleration)
        
    def acc_norm(self, u, t=0):
        """Norme de l'accélération appliquée au vecteur spécifié."""
        # Application de l'accélération au vecteur
        a = self.acceleration(u, t)
        # Retourne la norme du vecteur
        return np.linalg.norm(a)
    
    def __repr__(self):
        """Représentation de la force sous forme de string."""
        return self.name
    
    def _repr_latex_(self):
        """[Jupyter] Représentation de la force au format LaTeX."""
        return fr"{self.name} : $\displaystyle {self.formula}$"

Nous définissons également une classe `ForceSet` pour représenter un jeu de forces et son application à une particule donnée, ici la station spatiale internationnale.

À partir des définitions de classes des forces d'un jeu donné et de coordonnées initiales de position et de vitesse de l'ISS, `ForceSet` permet de calculer les positions et vitesses suivantes. Ce calcul se fait par résolution numérique de l'équation différentielle construite par la somme des accélération des forces impliquées. La méthode de Runge-Kutta d'ordre quatre (RK4) est utilisée dans cette résolution.

In [ ]:
import sys

class ForceSet():
    """Classe de jeu de forces à appliquer à une particule."""
    def __init__(self, forces):
        """Instancie la classe à partir d'une liste de forces."""
        self.forces = forces

    def derivee(self, u, t):
        """Calcule la dérivée à partir d'un vecteur de coordonnées.
        
        Dans un repère cartésien tridimensionnel, le vecteur de
        coordonnées est `[x, y, z, dx, dy, dz]`.
        """
        # Vecteur des coordonnées sources
        x, y, z, vx, vy, vz = u
        # Les vitesses sont les dérivées des positions
        dx, dy, dz = vx, vy, vz
        # Les acclélérations sont définies par les EDO de chaque force
        dvx, dvy, dvz = np.sum([force.acceleration(u, t)
                                for force in self.forces], axis=0)
        # Retourne les dérivées
        return np.array([dx, dy, dz, dvx, dvy, dvz])

    def magnitude(self, u):
        """Calcul des ordres de grandeur des forces à des coordonnées."""
        # Retourne un dictonnaire du log10 de chaque force aux coordonnées
        return {str(force): np.log10(force.acc_norm(u)) for force in forces}

    def solve(self, t_start, t_stop, t_step, coords, interrupt=None):
        """Résolution d'équation différentielle par la méthode de Runge-Kutta.

        Attributs:
            t_start:   valeur de départ de la variable temporelle
            t_stop:    valeur d'arrivée de la variable temporelle
            t_step:    intervalle de temps entre deux calculs
            coords:    coordonnées initiales
            interrupt: fonction pour interrompre prématurément le calcul (par
                       exemple, pour éviter des calculs inutiles une fois les
                       données hors-limites)
         """
        # Création du tableau temps
        num_points = int((t_stop - t_start) / t_step) + 1
        t = np.linspace(t_start, t_stop, num_points)
        # Initialisation du tableau solution
        v = np.empty((len(coords), num_points))
        # Condition initiale
        v[:, 0] = coords
        # Boucle for des variables de la méthode
        for i in range(num_points - 1):
            d1 = self.derivee(v[:, i], t[i])
            d2 = self.derivee(v[:, i] + t_step / 2 * d1, t[i] + t_step / 2)
            d3 = self.derivee(v[:, i] + t_step / 2 * d2, t[i] + t_step / 2)
            d4 = self.derivee(v[:, i] + t_step * d3,  t[i] + t_step)
            v[:, i + 1] = v[:, i] + t_step / 6 * (d1 + 2*d2 + 2*d3 + d4)
            if interrupt and interrupt(v[:, i+1]):
                t_inter = t_start + i*t_step
                print(f"Interruption du calcul de solution à t={t_inter}!",
                      file=sys.stderr)
                v[:,i+2:] = np.nan
                break
        # Retourne des tableaux des temps et des solutions
        return t, v

    def __repr__(self):
        """Représentation du jeu de forces sous forme de string."""
        return fr"ForceSet[{', '.join(str(force) for force in self.forces)}]"

## Ajout à la bibliothèque
Nous définissons ces deux classes dans la bibliothèque `isslib`, en tant que ressources du module principal.